# 3) Transforming data

Data in the `data/interim` is ready for further processing. Because the format is tabular, we can process it with `Pandas`. Before proceeding to data analysis, this data must be cleaned:

- From the dataset, discard offers that do not have the amount specified in `PLN`,
- Change the names of job offers and cities (`name` and `location_city` columns) to lower case,
- Add a new `salary_avg` column as an average of the `salary_high` and `salary_low` columns,
- Unify the names of cities:
    - e.g. `wroclove, wroclaw, wrocław` should be changed to `wrocław`,
    - e.g. `krakow, kraków +1, kraków +2` should be changed to `kraków`,
    - all other cases that appear should also be cleaned.
- `NoFluffJobs` does not share information on the country of employment so a column `location_city` needs to be created or completed:
    - if the work is remote, set `N/A`,
    - otherwise, unless the city name suggests otherwise, set `PL`,
- Check whether the `location_country` column is filled correctly
- Add a new column `is_senior`, that informs whether the position is a senior one or not. To do so use the name of the position: e.g. `Senior Data Analyst -> is_senior = 1`,
- Save the results to the `data\processed\ directory`, using the `;` separator, `UTF-8` encoding and without index (`index=False`). Take the same file name as in task 2 i.e. `job_offers_yyyy_mm_dd.csv`.

#### Hints:
- To unify the names of the cities, use the `unique()` method and manually correct the rows where the names are incorrect.
- To check the name of the position for the word "Senior", use regular expressions and the `re.match` method.


In [6]:
#import libraries
import pandas as pd
import re
import datetime

#load data
file_path = "data/interim/job_offers_2024_02_20.csv"

df = pd.read_csv(
  file_path,
  sep = ";",
  encoding = "UTF-8",
  decimal = "."
)

df.head(2)

,name,company,technology,job,location,salary
0,MD Senior Migration & Development Analyst - Re...,Link Group,"['SQL', 'VBA', 'Excel', 'MS Access']",data analyst,"{'city': 'Remote', 'country': 'N/A'}","{'low': 18500, 'high': 25500, 'currency': 'PLN'}"
1,Data Analyst (SAP),Team Connect Sp. z o.o.,"['SAP', 'Data models', 'SAP BW', 'Data visuali...",data analyst,"{'city': 'Remote', 'country': 'N/A'}","{'low': 23500, 'high': 28500, 'currency': 'PLN'}"


In [7]:
#columns salary, technology and location loaded from csv as string --> make objects out of it again (list/dictionary)

df["salary"] = [eval(i) for i in df["salary"]]
df["technology"] = [eval(i) for i in df["technology"]]
df["location"] = [eval(i) for i in df["location"]]

In [8]:
#identify indexes with the PLN currency
pln_indexes = []
for i, value in enumerate(df["salary"]):
    if df["salary"][i]["currency"] == "PLN":
        pln_indexes.append(i)

#filter offers with salary in PLN currency
dfPLN = df.iloc[pln_indexes,].reset_index()


#name and city in lower case
dfPLN["name"] = [i.lower() for i in dfPLN["name"]]

for i, value in enumerate(dfPLN["location"]):
    dfPLN["location"][i]["city"] = dfPLN["location"][i]["city"].lower()

In [9]:
#new column salary_avg

salary_avg = []
for i, value in enumerate(dfPLN["salary"]):
    avg = (dfPLN["salary"][i]["high"] + dfPLN["salary"][i]["low"]) / 2   #mean
    salary_avg.append(avg)

dfPLN["salary_avg"] = salary_avg

In [10]:
#unify the names of cities

cities = [dfPLN["location"][i]["city"] for i, value in enumerate(dfPLN["location"])]  #filter cities
cities = pd.Series(cities)
print(cities.unique())   #warsaw, warszawa

#replace directly at dictionary (column location)
for i, value in enumerate(dfPLN["location"]):
    if dfPLN["location"][i]["city"] == "warsaw":
        dfPLN["location"][i]["city"] = "warszawa"
        

['remote' 'gdańsk' 'kraków' 'warsaw' 'warszawa' 'wrocław' 'katowice'
 'poznań']


In [11]:
#new column location_city (anonymization)

dfPLN["location_city"] = cities

dfPLN.loc[dfPLN["location_city"] == "budapest", "location_city"] = "HU"
dfPLN.loc[dfPLN["location_city"] == "remote", "location_city"] = "N/A"
dfPLN.loc[(dfPLN["location_city"] != "N/A") & (dfPLN["location_city"] != "HU"), "location_city"] = "PL"

In [12]:
#new column location_country
country = [dfPLN["location"][i]["country"] for i in range(len(dfPLN["location"]))]  #zeme
country = pd.Series(country)

dfPLN["location_country"] = country
display(dfPLN.head(5))  #some N/A values where they shouldn't be

dfPLN["location_country"] = dfPLN["location_city"]   #add information from the location_city column

,index,name,company,technology,job,location,salary,salary_avg,location_city,location_country
0,0,md senior migration & development analyst - re...,Link Group,"[SQL, VBA, Excel, MS Access]",data analyst,"{'city': 'remote', 'country': 'N/A'}","{'low': 18500, 'high': 25500, 'currency': 'PLN'}",22000.0,N/A,N/A
1,1,data analyst (sap),Team Connect Sp. z o.o.,"[SAP, Data models, SAP BW, Data visualization]",data analyst,"{'city': 'remote', 'country': 'N/A'}","{'low': 23500, 'high': 28500, 'currency': 'PLN'}",26000.0,N/A,N/A
2,2,expert data analyst,emagine Sp. Z o.o.,"[Analyst, Business Analyst, SQL, SAS]",data analyst,"{'city': 'gdańsk', 'country': 'N/A'}","{'low': 25200, 'high': 28560, 'currency': 'PLN'}",26880.0,PL,N/A
3,3,quant analyst/model analyst,emagine Sp. Z o.o.,"[C++, Degree, PhD, MSc]",data analyst,"{'city': 'kraków', 'country': 'N/A'}","{'low': 23520, 'high': 23520, 'currency': 'PLN'}",23520.0,PL,N/A
4,4,lead bi analyst / data business analyst,EPAM Systems,"[Analyst, SQL, BI, Business Analyst]",data analyst,"{'city': 'warszawa', 'country': 'N/A'}","{'low': 22400, 'high': 25400, 'currency': 'PLN'}",23900.0,PL,N/A


In [14]:
#new column is_senior for identifying positions of seniors

names = dfPLN["name"]
regex = r"senior"

is_senior = []
for name in names:
    if re.search(regex,name):  #regex match found
        is_senior.append(1)
    else:
        is_senior.append(0)

dfPLN["is_senior"] = is_senior

dfPLN.head(5)

,index,name,company,technology,job,location,salary,salary_avg,location_city,location_country,is_senior
0,0,md senior migration & development analyst - re...,Link Group,"[SQL, VBA, Excel, MS Access]",data analyst,"{'city': 'remote', 'country': 'N/A'}","{'low': 18500, 'high': 25500, 'currency': 'PLN'}",22000.0,N/A,N/A,1
1,1,data analyst (sap),Team Connect Sp. z o.o.,"[SAP, Data models, SAP BW, Data visualization]",data analyst,"{'city': 'remote', 'country': 'N/A'}","{'low': 23500, 'high': 28500, 'currency': 'PLN'}",26000.0,N/A,N/A,0
2,2,expert data analyst,emagine Sp. Z o.o.,"[Analyst, Business Analyst, SQL, SAS]",data analyst,"{'city': 'gdańsk', 'country': 'N/A'}","{'low': 25200, 'high': 28560, 'currency': 'PLN'}",26880.0,PL,PL,0
3,3,quant analyst/model analyst,emagine Sp. Z o.o.,"[C++, Degree, PhD, MSc]",data analyst,"{'city': 'kraków', 'country': 'N/A'}","{'low': 23520, 'high': 23520, 'currency': 'PLN'}",23520.0,PL,PL,0
4,4,lead bi analyst / data business analyst,EPAM Systems,"[Analyst, SQL, BI, Business Analyst]",data analyst,"{'city': 'warszawa', 'country': 'N/A'}","{'low': 22400, 'high': 25400, 'currency': 'PLN'}",23900.0,PL,PL,0


In [15]:
#save to csv with current date

current_date = datetime.datetime.today().strftime('%Y_%m_%d')
file_name = f"job_offers_{current_date}.csv"

dfPLN.to_csv(r"data\processed\{}".format(file_name), sep = ";", header=True, index=False)